## 1. 패키지 로드

In [1]:
# 라이브러리 import
import os
import pandas as pd
import numpy as np
import math 

# 셀레늄
from selenium.webdriver.common.alert import Alert # 경고 처리
from selenium import webdriver  # 라이브러리(모듈) 가져오라
# pip install chromedriver_autoinstaller
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC # 상호작용 자동화
import chromedriver_autoinstaller

# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
from time import sleep
import time

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

## 2. Actor_namuwiki_clean.csv 로드

In [2]:
actors_df = pd.read_csv('Actor_namuwiki_clean.csv', encoding='utf-8-sig')

In [3]:
actors_df = actors_df.dropna(subset='Name')
actors_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820 entries, 0 to 819
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Name                820 non-null    object
 1   Age                 820 non-null    int64 
 2   activation_content  477 non-null    object
 3   Sex                 820 non-null    object
dtypes: int64(1), object(3)
memory usage: 25.8+ KB


## 3. 배우 크롤링 : 사진, 수상경력, 출연작품과 배역

In [4]:
name_list = actors_df.Name.to_list() # 배우 이름 리스트

In [5]:
name_list = name_list[527:]

In [6]:
# 크롬 옵션설정
options = webdriver.ChromeOptions()
options.add_argument("--window-size=800,1200") # 크롬 윈도우 사이즈 조절

# 크롬 load
chrome_path = chromedriver_autoinstaller.install()

In [7]:
actors_data = []

for name in tqdm_notebook(name_list):
    actor_dict = dict()
    driver = webdriver.Chrome(chrome_path, options=options)
    name = '배우' + ' ' + name
    
    driver.get("https://www.naver.com") # 네이버 접속

    # 네이버 검색어 입력 후 검색
    element = driver.find_element_by_name("query")
    element.clear()  # 혹시 검색창에 존재하는 텍스트 제거

    element.send_keys(name)  # 검색창에 검색어 전달
    element.submit()  # 검색 클릭
    time.sleep(1)
    
    # 프로필 클릭
    try:
        driver.find_element_by_link_text("프로필").click()
        time.sleep(1)
    except:
        print('프로필이 존재하지 않음.')
        try:
            driver.close()
        except:
            pass
        continue
        
    # 사진 url
    try:
        photo = driver.find_element_by_css_selector('div.cm_content_area._cm_content_area_profile > div > div.detail_info > a > img')
        photo_url = photo.get_attribute('src')
    except:
        photo_url = np.nan
    
    # 수상내역 더보기 클릭
    try:
        driver.find_element_by_css_selector(".cm_content_area._cm_content_area_prize .area_button_arrow").click()
        time.sleep(1)
    except:
        pass # 수상내역 더보기가 없으면 pass
    
    # 수상연도 + 내용
    prize_year_list = []
    prize_content_list = []

    for i in range(0, 100):  # 수상내역 페이지 최대 100페이지까지 수집할 수 있게 설정
        try:
            # 수상내역 크롤링
            prize = driver.find_element_by_css_selector(".cm_content_area._cm_content_area_prize .pro_info.date_type1").text
            # 년도만 세기
            p = re.compile('[0-9]{4,4}')   # 정규표현식으로 "0-9숫자 4자리" 수집
            m = p.findall(prize)           # prize 중에서 모두 찾아라(findall 함수)
            prize_year_list = prize_year_list + m    # 페이지 별 수상내역 리스트 전부 합치기
            prize_content_list = prize_content_list + prize.split('\n')[1::2]

            # 수상내역 다음페이지 클릭
            if driver.find_element_by_css_selector('.pg_next.on'):
                next = driver.find_element_by_css_selector('.pg_next.on').click()
                time.sleep(2)
        except:  
            break
    
    # 작품활동 - 방송 클릭
    try:
        driver.find_element_by_link_text("작품활동").click()
        time.sleep(2)
    except:
        pass
            
    try:
        film_num = int(driver.find_element_by_css_selector('.this_text .num').text)
        # 페이지 클릭횟수 계산
        page_click = math.ceil(film_num/4) - 1
    except:
        page_click = 0
        pass
    
    # 작품 데이터 수집
    title_list = [] # 제목
    main_sub_list = [] # 주연/조연
    cast_list = [] # 맡은 배역
    cast_link_list = [] # 맡은 배역의 설명 링크
    
    for i in range(page_click + 1):
        # 제목
        titles = driver.find_elements_by_css_selector('.this_text ._text')
            
        temp_list = []
        for title in titles:
            temp_list.append(title.text)
            temp_list = [x for x in temp_list if x !='' and x != '방송중']  
        title_list = title_list + temp_list
        
        # 주연/조연
        main_sub_temp = []
        for t in range(1, 5):
            try:
                main_sub = driver.find_element_by_css_selector('#mflick > div:nth-child({}) > div > div:nth-child({}) > div > div.data_box > span'.format(i+1, t)).text
                main_sub = main_sub[:2]
                main_sub_temp.append(main_sub)
            except:
                main_sub_temp.append(np.nan)
        main_sub_list = main_sub_list + main_sub_temp

        # 배역
        cast_temp = []
        for t in range(1, 5):
            try:
                cast = driver.find_element_by_css_selector('#mflick > div:nth-child({}) > div > div:nth-child({}) > div > div.data_box > span.sub_text'.format(i+1, t)).text
                cast = cast[3:]
                cast_temp.append(cast)
            except:
                cast_temp.append(np.nan)
        cast_list = cast_list + cast_temp

        # 배역 링크
        cast_link_temp = []
        for t in range(1, 5):
            try:
                cast = driver.find_element_by_css_selector('#mflick > div:nth-child({}) > div > div:nth-child({}) > div > div.data_box > span > a'.format(i+1, t))
                cast_link_temp.append(cast.get_attribute('href'))
            except:
                cast_link_temp.append(np.nan)
        cast_link_list = cast_link_list + cast_link_temp
    
        # 작품활동 다음페이지 클릭
        try:
            driver.find_element_by_css_selector('.pg_next.on').click()
            time.sleep(2)
        except:
            print('마지막 페이지')
            break
    
    # 배우별 데이터
    actor_prize_df = pd.DataFrame(list(zip(prize_year_list, prize_content_list)), columns=['수상연도', '수상내역'])
    
    # 배우 개인 데이터
    actor_work_activity_df = pd.DataFrame(list(zip(title_list, main_sub_list, cast_list, cast_link_list)),
                                         columns=['드라마 제목', '주연/조연', '배역', '배역설명url'])
    
    actor_dict['name'] = name.lstrip('배우')
    actor_dict['photo_url'] = photo_url
    actor_dict['prize_data'] = actor_prize_df
    actor_dict['work_data'] = actor_work_activity_df
    
    actors_data.append(actor_dict)
    
    print(actor_dict)
    
    driver.close()

  0%|          | 0/293 [00:00<?, ?it/s]

마지막 페이지
{'name': ' 박소현', 'photo_url': 'https://search.pstatic.net/common?type=b&size=216&expire=1&refresh=true&quality=100&direct=true&src=http%3A%2F%2Fsstatic.naver.net%2Fpeople%2Fportrait%2F202101%2F20210114174256728.jpg', 'prize_data':    수상연도                        수상내역
0  2016            SAF 연예대상 라디오 DJ상
1  2011             SBS 연예대상 라디오DJ상
2  2011  MBC 방송연예대상 쇼버라이어티부문 여자 우수상
3  2011               더 보이스 오브 SBS상
4  2010                  푸른미디어상 언어상
5  2009  제10회 대한민국영상대전 예능MC부문 포토제닉상
6  2007    SBS 방송연예대상 여자진행자부문 TV스타상
7  2004          SBS 연기대상 라디오부문 특별상, 'work_data':         드라마 제목 주연/조연 배역  배역설명url
0  지금부터 베리베리 해               NaN
1         비행소녀               NaN
2        비디오스타               NaN
3    공감다큐 해바라기               NaN}
마지막 페이지
{'name': ' 조', 'photo_url': nan, 'prize_data': Empty DataFrame
Columns: [수상연도, 수상내역]
Index: [], 'work_data': Empty DataFrame
Columns: [드라마 제목, 주연/조연, 배역, 배역설명url]
Index: []}
마지막 페이지
{'name': ' 박수진', 'photo_url': 'https://search.pstatic.net/common?ty

프로필이 존재하지 않음.
프로필이 존재하지 않음.
마지막 페이지
{'name': ' 박예린', 'photo_url': nan, 'prize_data': Empty DataFrame
Columns: [수상연도, 수상내역]
Index: [], 'work_data':     드라마 제목 주연/조연 배역  배역설명url
0  그린마더스클럽               NaN}
프로필이 존재하지 않음.
마지막 페이지
{'name': ' 박원숙', 'photo_url': 'https://search.pstatic.net/common?type=b&size=216&expire=1&refresh=true&quality=100&direct=true&src=http%3A%2F%2Fsstatic.naver.net%2Fpeople%2Fportrait%2F201911%2F20191115161235239-1399700.jpg', 'prize_data':    수상연도                      수상내역
0  2013              MBC 연기대상 공로상
1  2007     MBC 연기대상 중견배우부문 황금연기상
2  2005  제12회 대한민국연예예술상 문화관광부장관표창
3  1998            MBC 연기대상 여자우수상
4  1989               한국연극영화상 신인상, 'work_data':               드라마 제목 주연/조연     배역  \
0            구필수는 없다    천만      역   
1                 마인    조연  양순혜 역   
2   박원숙의 같이 삽시다 시...                
3        박원숙의 같이 삽시다                
4              모던패밀리                
..               ...   ...    ...   
63           한지붕 세가족   NaN    NaN   
64             엄마의 

마지막 페이지
{'name': ' 박지예', 'photo_url': 'https://search.pstatic.net/common?type=b&size=216&expire=1&refresh=true&quality=100&direct=true&src=http%3A%2F%2Fsstatic.naver.net%2Fpeople%2Fportrait%2F202111%2F20211123140933432.jpg', 'prize_data': Empty DataFrame
Columns: [수상연도, 수상내역]
Index: [], 'work_data':                드라마 제목 주연/조연   배역  \
0              어사와 조이   NaN  NaN   
1  무브 투 헤븐 : 나는 유품...   NaN  NaN   
2         브람스를 좋아하세요?   NaN  NaN   
3            연애공식 구하리    구하    역   
4             어쨌든 기념일    김완    역   
5            기억, 하리 2   NaN  NaN   
6              기억, 하리   NaN  NaN   

                                             배역설명url  
0                                                NaN  
1                                                NaN  
2                                                NaN  
3  https://search.naver.com/search.naver?where=ne...  
4  https://search.naver.com/search.naver?where=ne...  
5                                                NaN  
6                        

마지막 페이지
{'name': ' 박희본', 'photo_url': 'https://search.pstatic.net/common?type=b&size=216&expire=1&refresh=true&quality=100&direct=true&src=http%3A%2F%2Fsstatic.naver.net%2Fpeople%2F92%2F202204111107391001.jpg', 'prize_data': Empty DataFrame
Columns: [수상연도, 수상내역]
Index: [], 'work_data':                 드라마 제목 주연/조연        배역  \
0                 해피니스    조연     나현경 역   
1   (아는 건 별로 없지만) 가...                   
2              시크릿 부티크    주연     위예남 역   
3               아는 와이프    조연     차주은 역   
4     드라마 스테이지 - 오늘...    주연     오문숙 역   
5           그냥 사랑하는 사이    조연     김완진 역   
6                  아르곤    조연     육혜리 역   
7            초인가족 2017    조연  안정민 대리 역   
8                  도깨비   NaN       NaN   
9           출출한 여자 시즌2   NaN       NaN   
10              대세는 백합   NaN       NaN   
11                 풍선껌   NaN       NaN   
12        내친구와 식샤를 합시다   NaN       NaN   
13                프로듀사   NaN       NaN   
14          운명처럼 널 사랑해   NaN       NaN   
15   드라마 스페셜 - 내가 결...   NaN       NaN   

...

마지막 페이지
{'name': ' 표예진', 'photo_url': nan, 'prize_data':    수상연도                          수상내역
0  2019    SBS 연기대상 여자 베스트 캐릭터상 (VIP)
1  2018          아시아모델어워즈 뉴스타상 연기자 부문
2  2016  제24회 대한민국문화연예대상 드라마부문 여자 신인상, 'work_data':          드라마 제목 주연/조연         배역  \
0          모범택시    주연      안고은 역   
1           VIP    주연      온유리 역   
2       팔로우미 11                    
3    김비서가 왜 그럴까    조연      김지아 역   
4       미워도 사랑해    주연      길은조 역   
5       쌈, 마이웨이    조연      장예진 역   
6   월계수 양복점 신사들    조연      김다정 역   
7           닥터스    조연  현수진 간호사 역   
8      72초 시즌 3   NaN        NaN   
9          결혼계약   NaN        NaN   
10      두여자 시즌1   NaN        NaN   

                                              배역설명url  
0   https://search.naver.com/search.naver?where=ne...  
1   https://search.naver.com/search.naver?where=ne...  
2                                                 NaN  
3   https://search.naver.com/search.naver?where=ne...  
4   https://search.naver.com/search.naver?where=ne...  
5   http

프로필이 존재하지 않음.
프로필이 존재하지 않음.
마지막 페이지
{'name': ' 한보름', 'photo_url': 'https://search.pstatic.net/common?type=b&size=216&expire=1&refresh=true&quality=100&direct=true&src=http%3A%2F%2Fsstatic.naver.net%2Fpeople%2Fportrait%2F202111%2F20211115142719734.jpg', 'prize_data':    수상연도                          수상내역
0  2018    제13회 아시아 모델 어워즈 연기자부문 뉴스타상
1  2017  제25회 대한민국 문화연예대상 드라마부문 여자신인상, 'work_data':               드라마 제목 주연/조연     배역  \
0            트렌드 레코드                
1           오! 삼광빌라!    주연  장서아 역   
2            와이낫 시즌3                
3                타이밍                
4       정글의 법칙 IN 코론   NaN    NaN   
5     수작남녀-CRAFTSMAN   NaN    NaN   
6      정글의 법칙 in 메르귀   NaN    NaN   
7                레벨업    주연  신연화 역   
8       백년만의 귀향, 집으로   NaN    NaN   
9          호구들의 감빵생활   NaN    NaN   
10           바람난 언니들   NaN    NaN   
11   정글의 법칙 in 북마리아나   NaN    NaN   
12       알함브라 궁전의 추억   NaN    NaN   
13             독립한보름   NaN    NaN   
14              사다드림   NaN    NaN   
15   Beauty's

마지막 페이지
{'name': ' 한혜숙', 'photo_url': 'https://search.pstatic.net/common?type=b&size=216&expire=1&refresh=true&quality=100&direct=true&src=http%3A%2F%2Fsstatic.naver.net%2Fpeople%2F153%2F20070102114159197095472.jpg', 'prize_data':    수상연도              수상내역
0  2010     불기 2554년 불자대상
1  2554  SBS 연기대상 10대 스타상
2  2006       SBS 연기대상 대상
3  2006     한국방송대상 TV연기자상
4  1987    KBS 우수프로그램 연기상
5  1987    한국백상예술대상 TV연기상, 'work_data':         드라마 제목 주연/조연     배역                                            배역설명url
0        보석비빔밥    주연  피혜자 역  https://search.naver.com/search.naver?where=ne...
1        하늘이시여    주연  지영선 역  https://search.naver.com/search.naver?where=ne...
2       왕꽃 선녀님                                                             NaN
3        인어아가씨    조연  심수정 역  https://search.naver.com/search.naver?where=ne...
4     색소폰과 찹쌀떡                                                             NaN
5           미나                                                             NaN
6         왕과 비      

마지막 페이지
{'name': ' 홍세', 'photo_url': nan, 'prize_data': Empty DataFrame
Columns: [수상연도, 수상내역]
Index: [], 'work_data':     드라마 제목  주연/조연  배역  배역설명url
0   부사덕적미소    NaN NaN      NaN
1  용감설출아애니    NaN NaN      NaN
2  아요변성경시자    NaN NaN      NaN}
마지막 페이지
{'name': ' 홍수아', 'photo_url': nan, 'prize_data': Empty DataFrame
Columns: [수상연도, 수상내역]
Index: [], 'work_data': Empty DataFrame
Columns: [드라마 제목, 주연/조연, 배역, 배역설명url]
Index: []}
마지막 페이지
{'name': ' 홍수현', 'photo_url': 'https://search.pstatic.net/common?type=b&size=216&expire=1&refresh=true&quality=100&direct=true&src=http%3A%2F%2Fsstatic.naver.net%2Fpeople%2Fportrait%2F202005%2F20200508101215464.jpg', 'prize_data':    수상연도                     수상내역
0  2013  MBC 연기대상 연속극부문 여자 우수연기상
1  2011  KBS 연기대상 중편드라마부문 여자 우수상
2  2011          KBS 연기대상 베스트커플상
3  2005       KBS 연기대상 여자 특집단막극상
4  2003             KBS 연기대상 조연상
5  2001          KBS 연기대상 신인 연기상, 'work_data':                드라마 제목 주연/조연     배역  \
0                올리뷰쇼                
1            

마지막 페이지
{'name': ' 황영희', 'photo_url': 'https://search.pstatic.net/common?type=b&size=216&expire=1&refresh=true&quality=100&direct=true&src=http%3A%2F%2Fsstatic.naver.net%2Fpeople%2Fportrait%2F201409%2F20140917154539649-566301.png', 'prize_data': Empty DataFrame
Columns: [수상연도, 수상내역]
Index: [], 'work_data':                드라마 제목 주연/조연     배역  \
0              오늘의 웹툰                
1             징크스의 연인    조연  방여사 역   
2              원 더 우먼    조연  강은화 역   
3              오월의 청춘    조연  최순녀 역   
4             달이 뜨는 강    조연   사씨 역   
5      제발 그 남자 만나지 마요                
6            그놈이 그놈이다    조연  정영순 역   
7        더 킹 : 영원의 군주    조연  민화연 역   
8             두 번은 없다   NaN    NaN   
9            동백꽃 필 무렵   NaN    NaN   
10             호텔 델루나   NaN    NaN   
11                녹두꽃   NaN    NaN   
12               사의찬미   NaN    NaN   
13             황후의 품격   NaN    NaN   
14              계룡선녀전   NaN    NaN   
15           내 사랑 치유기   NaN    NaN   
16         추리의 여왕 시즌2   NaN    NaN   
17  드라마 

천호진에서 멈춤(357번)
박소현에서 멈춤(

### 5. Actor Data => one data Frame

In [8]:
actors_data[0]['work_data'].dropna()

,드라마 제목,주연/조연,배역,배역설명url


In [9]:
data_dict = {}
cnt = 0

for data in actors_data:
    target_info = {}
    
    name = data['name']
    photo_url = data['photo_url']
    prize_df = data['prize_data'].dropna()
    work_df = data['work_data'].dropna()
    
    # prize_df -> one list
    prize_years_list = prize_df.수상연도.to_list()
    prize_content_list = prize_df.수상내역.to_list()
    prize_list = []
    
    for i in list(zip(prize_years_list, prize_content_list)):
        prize_list.append(' '.join(i))
    
    # work_df -> one list
    drama_title_list = work_df['드라마 제목'].to_list()
    drama_cast_kind_list = work_df['주연/조연'].to_list()
    drama_cast_list = work_df['배역'].to_list()
    drama_cast_info_url_list = work_df['배역설명url'].to_list()
    
    drama_list = []
    for i in list(zip(drama_title_list, drama_cast_kind_list, drama_cast_list, drama_cast_info_url_list)):
        drama_list.append('/'.join(i))
    
    target_info['이름'] = name
    target_info['사진url'] = photo_url
    target_info['수상내역'] = prize_list
    target_info['드라마활동'] = drama_list
    
    data_dict[cnt] = target_info
    cnt = cnt + 1

In [10]:
result = pd.DataFrame.from_dict(data_dict, orient='index')
result.head()

,이름,사진url,수상내역,드라마활동
0,박소현,https://search.pstatic.net/common?type=b&size=...,"[2016 SAF 연예대상 라디오 DJ상, 2011 SBS 연예대상 라디오DJ상, ...",[]
1,조,NaN,[],[]
2,박수진,https://search.pstatic.net/common?type=b&size=...,[2014 서울특별시장상],[칼과 꽃/주연/모설 역/https://search.naver.com/search....
3,박시은,https://search.pstatic.net/common?type=b&size=...,[],[모두 다 쿵따리/주연/송보미 역/https://search.naver.com/se...
4,박신혜,NaN,"[2017 아시아 아티스트 어워즈 아시아 아이콘상, 2016 SBS 연기대상 장르&...",[시지프스 : the myth/주연/강서해 역/https://search.naver...


In [11]:
result['이름'] = result.이름.apply(lambda x : x.strip())

In [12]:
result.info() # 사진에 널값이 존재

<class 'pandas.core.frame.DataFrame'>
Int64Index: 223 entries, 0 to 222
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   이름      223 non-null    object
 1   사진url   169 non-null    object
 2   수상내역    223 non-null    object
 3   드라마활동   223 non-null    object
dtypes: object(4)
memory usage: 8.7+ KB


In [13]:
# 널값 제거
result = result.dropna().reset_index(drop=True)

In [14]:
def clean_result(x):
    if x:
        pass
    else:
        x = np.nan
    return x

result['드라마활동'] = result.드라마활동.apply(clean_result)
result['수상내역'] = result.수상내역.apply(clean_result)

In [15]:
# 널값 제거
result = result.dropna().reset_index(drop=True)

In [16]:
result = result.rename(columns={'이름' : 'Name'})

In [17]:
result.head(5)

,Name,사진url,수상내역,드라마활동
0,박수진,https://search.pstatic.net/common?type=b&size=...,[2014 서울특별시장상],[칼과 꽃/주연/모설 역/https://search.naver.com/search....
1,박원숙,https://search.pstatic.net/common?type=b&size=...,"[2013 MBC 연기대상 공로상, 2007 MBC 연기대상 중견배우부문 황금연기상...",[구필수는 없다/천만/ 역/https://search.naver.com/search...
2,박은빈,https://search.pstatic.net/common?type=b&size=...,"[2021 KBS 연기대상 여자 최우수상 (연모), 2021 KBS 연기대상 여자 ...",[이상한 변호사 우영우/주연/우영우 역/https://search.naver.com...
3,박지영,https://search.pstatic.net/common?type=b&size=...,"[2012 제21회 부일영화상 여우조연상, 1995 문화체육관광부장관표창]",[현재는 아름다워/조연/진수정 역/https://search.naver.com/se...
4,박하나,https://search.pstatic.net/common?type=b&size=...,"[2021 KBS 연기대상 장편드라마부문 여자 우수상 (신사와 아가씨), 2020 ...",[신사와 아가씨/주연/조사라 역/https://search.naver.com/sea...


In [18]:
actors_df.head(5)

,Name,Age,activation_content,Sex
0,감우성,52,"MBC 20기 공채 탤런트로 배우 생활을 시작했으며 동기로 한석규, 박철, 차광수,...",남
1,강경준,39,"초등학교 시절에는 야구, 중학교 시절에는 농구선수였었다. 농구를 소재로 한 드라마 ...",남
2,강기영,39,"tvN 드라마 고교처세왕, 오 나의 귀신님에서 감초역할로 등장하며 이름을 알렸다. ...",남
3,강남길,64,"서울수색초등학교, 서울양정중학교, 서울대신고등학교를 졸업했고 한양대학교 영어영문학과...",남
4,강동원,41,학창 시절 (강동원 졸업사진)중학교 때는 전교 2등을 할 정도로 공부를 굉장히 잘했...,남


In [19]:
result2 = pd.merge(result, actors_df, how='inner', on='Name')
result2 = result2.dropna().reset_index(drop=True)
result2.head()

,Name,사진url,수상내역,드라마활동,Age,activation_content,Sex
0,박수진,https://search.pstatic.net/common?type=b&size=...,[2014 서울특별시장상],[칼과 꽃/주연/모설 역/https://search.naver.com/search....,37,슈가를 탈퇴한 뒤 새로운 소속사의 방침에 따라 연기 공부를 시작했으나 기회를 많이 ...,여
1,박원숙,https://search.pstatic.net/common?type=b&size=...,"[2013 MBC 연기대상 공로상, 2007 MBC 연기대상 중견배우부문 황금연기상...",[구필수는 없다/천만/ 역/https://search.naver.com/search...,73,1970년 MBC 2기 공채 탤런트에 합격하며 데뷔했다.사실 애초에 1969년 1기...,여
2,박지영,https://search.pstatic.net/common?type=b&size=...,"[2012 제21회 부일영화상 여우조연상, 1995 문화체육관광부장관표창]",[현재는 아름다워/조연/진수정 역/https://search.naver.com/se...,53,"전주우석여자고등학교[5] 재학 시절 플루트를 전공하여 음대를 지망하였으나, 재수생이...",여
3,박하나,https://search.pstatic.net/common?type=b&size=...,"[2021 KBS 연기대상 장편드라마부문 여자 우수상 (신사와 아가씨), 2020 ...",[신사와 아가씨/주연/조사라 역/https://search.naver.com/sea...,37,오랜 무명 생활 끝에 2014년 MBC 일일 드라마 압구정 백야에서 가족을 버린 친...,여
4,박한별,https://search.pstatic.net/common?type=b&size=...,"[2017 MBC 방송연예대상 쇼 시트콤 여자 우수상, 2015 SBS 연기대상 장...",[슬플 때 사랑한다/주연/윤마리 역/https://search.naver.com/s...,38,"원조 얼짱이었다. 안양예고 시절부터 구혜선, 남상미[6], 이주연 등과 함께 1기 ...",여


In [20]:
result2 = result2[['Name', 'Age', 'Sex', '수상내역', '드라마활동', 'activation_content', '사진url']]
result2

,Name,Age,Sex,수상내역,드라마활동,activation_content,사진url
0,박수진,37,여,[2014 서울특별시장상],[칼과 꽃/주연/모설 역/https://search.naver.com/search....,슈가를 탈퇴한 뒤 새로운 소속사의 방침에 따라 연기 공부를 시작했으나 기회를 많이 ...,https://search.pstatic.net/common?type=b&size=...
1,박원숙,73,여,"[2013 MBC 연기대상 공로상, 2007 MBC 연기대상 중견배우부문 황금연기상...",[구필수는 없다/천만/ 역/https://search.naver.com/search...,1970년 MBC 2기 공채 탤런트에 합격하며 데뷔했다.사실 애초에 1969년 1기...,https://search.pstatic.net/common?type=b&size=...
2,박지영,53,여,"[2012 제21회 부일영화상 여우조연상, 1995 문화체육관광부장관표창]",[현재는 아름다워/조연/진수정 역/https://search.naver.com/se...,"전주우석여자고등학교[5] 재학 시절 플루트를 전공하여 음대를 지망하였으나, 재수생이...",https://search.pstatic.net/common?type=b&size=...
3,박하나,37,여,"[2021 KBS 연기대상 장편드라마부문 여자 우수상 (신사와 아가씨), 2020 ...",[신사와 아가씨/주연/조사라 역/https://search.naver.com/sea...,오랜 무명 생활 끝에 2014년 MBC 일일 드라마 압구정 백야에서 가족을 버린 친...,https://search.pstatic.net/common?type=b&size=...
4,박한별,38,여,"[2017 MBC 방송연예대상 쇼 시트콤 여자 우수상, 2015 SBS 연기대상 장...",[슬플 때 사랑한다/주연/윤마리 역/https://search.naver.com/s...,"원조 얼짱이었다. 안양예고 시절부터 구혜선, 남상미[6], 이주연 등과 함께 1기 ...",https://search.pstatic.net/common?type=b&size=...
...,...,...,...,...,...,...,...
82,한채아,40,여,"[2015 KBS 연기대상 일일극부문 여자 우수 연기상, 2015 KBS 연기대상 ...",[연모/빈궁/역/https://search.naver.com/search.naver...,부산 출생이지만 울산에서 성장했다. 때문인지 예능에서 김태희와 함께 일명 울산 5대...,https://search.pstatic.net/common?type=b&size=...
83,한채영,42,여,"[2017 MBC 방송연예대상 버라이어티 여자 신인상, 2011 드라마 어워즈 온라...",[스폰서/주연/한채린 역/https://search.naver.com/search....,인천에서 태어나 대구에서 유년기를 보내다가 중학교 때 가족을 따라 미국으로 이민을 ...,https://search.pstatic.net/common?type=b&size=...
84,한혜숙,71,여,"[2010 불기 2554년 불자대상, 2554 SBS 연기대상 10대 스타상, 20...",[보석비빔밥/주연/피혜자 역/https://search.naver.com/searc...,"젊은 시절원래는 1970년 MBC 2기 공채 탤런트로 데뷔했으나[3], 탤런트 연수...",https://search.pstatic.net/common?type=b&size=...
85,한혜진,41,여,"[2013 제50회 저축의 날 대통령 표창, 2012 SBS 연예대상 토크쇼부문 우...",[외출/주연/한정은 역/https://search.naver.com/search.n...,"2002년 원빈, 후카다 쿄코 주연의 한일 합작 드라마 프렌즈로 데뷔. 이후 MBC...",https://search.pstatic.net/common?type=b&size=...


### 배역 내용 가져오기

In [26]:
load_df = pd.read_csv('test_actors_info.csv', encoding='utf-8-sig')
load_df

,Name,Age,Sex,수상내역,드라마활동,activation_content,사진url
0,감우성,52,남,"['2018 SBS 연기대상 대상', '2018 SBS 연기대상 베스트 커플상', ...",['바람이 분다/주연/권도훈 역/https://search.naver.com/sea...,"MBC 20기 공채 탤런트로 배우 생활을 시작했으며 동기로 한석규, 박철, 차광수,...",https://search.pstatic.net/common?type=b&size=...
1,강경준,39,남,"['2017 MBC 연기대상 연속극부문 남자 우수연기상', '2004 MBC 방송연...",['별별 며느리/주연/최한주 역/https://search.naver.com/sea...,"초등학교 시절에는 야구, 중학교 시절에는 농구선수였었다. 농구를 소재로 한 드라마 ...",https://search.pstatic.net/common?type=b&size=...
2,강기영,39,남,"['2019 제8회 대한민국 베스트 스타상 베스트 조연상', '2019 제12회 코...",['이상한 변호사 우영우/주연/정명석 역/https://search.naver.co...,"tvN 드라마 고교처세왕, 오 나의 귀신님에서 감초역할로 등장하며 이름을 알렸다. ...",https://search.pstatic.net/common?type=b&size=...
3,강남길,64,남,['2009 MBC 연기대상 황금연기상 중견배우 부문'],['왼손잡이 아내/조연/오창수 역/https://search.naver.com/se...,"서울수색초등학교, 서울양정중학교, 서울대신고등학교를 졸업했고 한양대학교 영어영문학과...",https://search.pstatic.net/common?type=b&size=...
4,강지섭,41,남,['2006 SBS 연기대상 뉴스타상'],['브라보 마이 라이프/주연/설도현 역/https://search.naver.com...,고향이 부산이며 70만원을 들고 서울로 상경해 고시원에서 생활을 했다. 어렸을 때부...,https://search.pstatic.net/common?type=b&size=...
...,...,...,...,...,...,...,...
251,한채아,40,여,"['2015 KBS 연기대상 일일극부문 여자 우수 연기상', '2015 KBS 연기...",['연모/빈궁/역/https://search.naver.com/search.nave...,부산 출생이지만 울산에서 성장했다. 때문인지 예능에서 김태희와 함께 일명 울산 5대...,https://search.pstatic.net/common?type=b&size=...
252,한채영,42,여,"['2017 MBC 방송연예대상 버라이어티 여자 신인상', '2011 드라마 어워즈...",['스폰서/주연/한채린 역/https://search.naver.com/search...,인천에서 태어나 대구에서 유년기를 보내다가 중학교 때 가족을 따라 미국으로 이민을 ...,https://search.pstatic.net/common?type=b&size=...
253,한혜숙,71,여,"['2010 불기 2554년 불자대상', '2554 SBS 연기대상 10대 스타상'...",['보석비빔밥/주연/피혜자 역/https://search.naver.com/sear...,"젊은 시절원래는 1970년 MBC 2기 공채 탤런트로 데뷔했으나[3], 탤런트 연수...",https://search.pstatic.net/common?type=b&size=...
254,한혜진,41,여,"['2013 제50회 저축의 날 대통령 표창', '2012 SBS 연예대상 토크쇼부...",['외출/주연/한정은 역/https://search.naver.com/search....,"2002년 원빈, 후카다 쿄코 주연의 한일 합작 드라마 프렌즈로 데뷔. 이후 MBC...",https://search.pstatic.net/common?type=b&size=...


In [32]:
load_df[load_df.Age == 1978]['Age'] = 2022 - load_df[load_df.Age == 1978].Age
load_df[load_df.Age == 1978]

,Name,Age,Sex,수상내역,드라마활동,activation_content,사진url
79,유인촌,1978,남,"['2022 서울문화재단 특별공로상', '2008 제17회 일본영화비평가대상 국제협...",['백중사 이야기///https://search.naver.com/search.na...,1951년 3월 20일 전라북도 완주군에서 태어나 서울에서 어린 시절을 보냈다. 한...,https://search.pstatic.net/common?type=b&size=...


In [22]:
result2.head(2)

,Name,Age,Sex,수상내역,드라마활동,activation_content,사진url
0,박수진,37,여,[2014 서울특별시장상],[칼과 꽃/주연/모설 역/https://search.naver.com/search....,슈가를 탈퇴한 뒤 새로운 소속사의 방침에 따라 연기 공부를 시작했으나 기회를 많이 ...,https://search.pstatic.net/common?type=b&size=...
1,박원숙,73,여,"[2013 MBC 연기대상 공로상, 2007 MBC 연기대상 중견배우부문 황금연기상...",[구필수는 없다/천만/ 역/https://search.naver.com/search...,1970년 MBC 2기 공채 탤런트에 합격하며 데뷔했다.사실 애초에 1969년 1기...,https://search.pstatic.net/common?type=b&size=...


In [23]:
result = pd.concat([load_df, result2])
result

,Name,Age,Sex,수상내역,드라마활동,activation_content,사진url
0,감우성,52,남,"['2018 SBS 연기대상 대상', '2018 SBS 연기대상 베스트 커플상', ...",['바람이 분다/주연/권도훈 역/https://search.naver.com/sea...,"MBC 20기 공채 탤런트로 배우 생활을 시작했으며 동기로 한석규, 박철, 차광수,...",https://search.pstatic.net/common?type=b&size=...
1,강경준,39,남,"['2017 MBC 연기대상 연속극부문 남자 우수연기상', '2004 MBC 방송연...",['별별 며느리/주연/최한주 역/https://search.naver.com/sea...,"초등학교 시절에는 야구, 중학교 시절에는 농구선수였었다. 농구를 소재로 한 드라마 ...",https://search.pstatic.net/common?type=b&size=...
2,강기영,39,남,"['2019 제8회 대한민국 베스트 스타상 베스트 조연상', '2019 제12회 코...",['이상한 변호사 우영우/주연/정명석 역/https://search.naver.co...,"tvN 드라마 고교처세왕, 오 나의 귀신님에서 감초역할로 등장하며 이름을 알렸다. ...",https://search.pstatic.net/common?type=b&size=...
3,강남길,64,남,['2009 MBC 연기대상 황금연기상 중견배우 부문'],['왼손잡이 아내/조연/오창수 역/https://search.naver.com/se...,"서울수색초등학교, 서울양정중학교, 서울대신고등학교를 졸업했고 한양대학교 영어영문학과...",https://search.pstatic.net/common?type=b&size=...
4,강지섭,41,남,['2006 SBS 연기대상 뉴스타상'],['브라보 마이 라이프/주연/설도현 역/https://search.naver.com...,고향이 부산이며 70만원을 들고 서울로 상경해 고시원에서 생활을 했다. 어렸을 때부...,https://search.pstatic.net/common?type=b&size=...
...,...,...,...,...,...,...,...
82,한채아,40,여,"[2015 KBS 연기대상 일일극부문 여자 우수 연기상, 2015 KBS 연기대상 ...",[연모/빈궁/역/https://search.naver.com/search.naver...,부산 출생이지만 울산에서 성장했다. 때문인지 예능에서 김태희와 함께 일명 울산 5대...,https://search.pstatic.net/common?type=b&size=...
83,한채영,42,여,"[2017 MBC 방송연예대상 버라이어티 여자 신인상, 2011 드라마 어워즈 온라...",[스폰서/주연/한채린 역/https://search.naver.com/search....,인천에서 태어나 대구에서 유년기를 보내다가 중학교 때 가족을 따라 미국으로 이민을 ...,https://search.pstatic.net/common?type=b&size=...
84,한혜숙,71,여,"[2010 불기 2554년 불자대상, 2554 SBS 연기대상 10대 스타상, 20...",[보석비빔밥/주연/피혜자 역/https://search.naver.com/searc...,"젊은 시절원래는 1970년 MBC 2기 공채 탤런트로 데뷔했으나[3], 탤런트 연수...",https://search.pstatic.net/common?type=b&size=...
85,한혜진,41,여,"[2013 제50회 저축의 날 대통령 표창, 2012 SBS 연예대상 토크쇼부문 우...",[외출/주연/한정은 역/https://search.naver.com/search.n...,"2002년 원빈, 후카다 쿄코 주연의 한일 합작 드라마 프렌즈로 데뷔. 이후 MBC...",https://search.pstatic.net/common?type=b&size=...


In [24]:
result.to_csv('test_actors_info.csv', encoding='utf-8-sig', index=False)